#14 Scripts

There are two main ways to add scripting for Pyomo models: using Python scripts and using callbacks for the <span style="color:darkblue; font-family:Courier">pyomo</span> command that alter or supplement its workflow.

>#####Note
>The examples are written to conform with the Python version 3 print function. If executed with Python version 2, the output from print statements may not look as nice.

###14.1. Python Scripts
#####14.1.1. Iterative Example

To illustrate Python scripts for Pyomo we consider an example that is in the file <span style="color:darkblue; font-family:Courier">iterative1.py</span> and is executed using the command

In [ ]:
!python iterative1.py

>#####Note
>This is a Python script that contains elements of Pyomo, so it is executed using the python command. The pyomo command can be used, but then there will be some strange messages at the end when Pyomo finishes the script and attempts to send the results to a solver, which is what the <span style="color:darkblue; font-family:Courier">pyomo</span> command does.

This script creates a model, solves it, and then adds a constraint to preclude the solution just found. This process is repeated, so the script finds and prints multiple solutions. The particular model it creates is just the sum of four binary variables. One does not need a computer to solve the problem or even to iterate over solutions. This example is provided just to illustrate some elementary aspects of scripting.

>#####Note
>	The built-in code for printing solutions prints only non-zero variable values. So if you run this code, no variable values will be output for the first solution found because all of the variables are zero. However, other information about the solution, such as the objective value, will be displayed.


In [ ]:
# iterative1.py

from pyomo.environ import *
from pyomo.opt import SolverFactory

# Create a solver
opt = SolverFactory('glpk')

#
# A simple model with binary variables and
# an empty constraint list.
#
model = AbstractModel()
model.n = Param(default=4)
model.x = Var(RangeSet(model.n), within=Binary)
def o_rule(model):
    return summation(model.x)
model.o = Objective(rule=o_rule)
model.c = ConstraintList()

# Create a model instance and optimize
instance = model.create_instance()
results = opt.solve(instance)
print(results)

# Iterate to eliminate the previously found solution
for i in range(5):
    instance.solutions.load_from(results)

    expr = 0
    for j in instance.x:
        if instance.x[j].value == 0:
            expr += instance.x[j]
        else:
            expr += (1-instance.x[j])
    instance.c.add( expr >= 1 )

    results = opt.solve(instance)
    print(results)

Let us now analyze this script. The first line is a comment that happens to give the name of the file. This is followed by two lines that import symbols for Pyomo:



In [ ]:
# iterative1.py
from pyomo.environ import *
from pyomo.opt import SolverFactory

An object to perform optimization is created by calling <span style="color:darkblue; font-family:Courier">SolverFactory</span> with an argument giving the name of the solver.t The argument would be <span style="color:darkblue; font-family:Courier">*gurobi*</span> if, e.g., Gurobi was desired instead of glpk:

In [ ]:
# Create a solver
opt = SolverFactory('glpk')

The next lines after a comment create a model. For our discussion here, we will refer to this as the base model because it will be extended by adding constraints later. (The words "base model" are not reserved words, they are just being introduced for the discussion of this example). There are no constraints in the base model, but that is just to keep it simple. Constraints could be present in the base model. Even though it is an abstract model, the base model is fully specified by these commands because it requires no external data:



In [ ]:
model = AbstractModel()
model.n = Param(default=4)
model.x = Var(RangeSet(model.n), within=Binary)
def o_rule(model):
    return summation(model.x)
model.o = Objective(rule=o_rule)

The next line is not part of the base model specification. It creates an empty constraint list that the script will use to add constraints.



In [ ]:
model.c = ConstraintList()

The next non-comment line creates the instantiated model and refers to the instance object with a Python variable <span style="color:darkblue; font-family:Courier">instance</span>. Models run using the <span style="color:darkblue; font-family:Courier">pyomo</span> script do not typically contain this line because model instantiation is done by the <span style="color:darkblue; font-family:Courier">pyomo</span> script. In this example, the <span style="color:darkblue; font-family:Courier">create</span> function is called without arguments because none are needed; however, the name of a file with data commands is given as an argument in many scripts.



In [ ]:
instance = model.create_instance()

The next line invokes the solver and refers to the object contain results with the Python variable <span style="color:darkblue; font-family:Courier">results</span>.

In [ ]:
results = opt.solve(instance)

The print method of the results object is invoked by the Python <span style="color:darkblue; font-family:Courier">print</span> command:

In [ ]:
print(results)

The next non-comment line is a Python iteration command that will successively assign the integers from 0 to 4 to the Python variable i, although that variable is not used in script. This loop is what causes the script to generate five more solutions:



In [ ]:
for i in range(5):

The next line associates the results obtained with the instance. This then enables direct queries of solution values in subsequent lines using variable names contained in the instance:

In [ ]:
    instance.solutions.load_from(results)

An expression is built up in the Python variable named <span style="color:darkblue; font-family:Courier">expr</span>. The Python variable <span style="color:darkblue; font-family:Courier">j</span> will be iteratively assigned all of the indexes of the variable <span style="color:darkblue; font-family:Courier">x</span>. For each index, the value of the variable (which was loaded by the <span style="color:darkblue; font-family:Courier">load</span> method just described) is tested to see if it is zero and the expression in <span style="color:darkblue; font-family:Courier">expr</span> is augmented accordingly. Although <span style="color:darkblue; font-family:Courier">expr</span> is initialized to 0 (an integer), its type will change to be a Pyomo expression when it is assigned expressions involving Pyomo variable objects:



In [ ]:
    expr = 0
    for j in instance.x:
        if instance.x[j].value == 0:
            expr += instance.x[j]
        else:
            expr += (1-instance.x[j])

During the first iteration (when <span style="color:darkblue; font-family:Courier">i</span> is 0), we know that all values of <span style="color:darkblue; font-family:Courier">x</span> will be 0, so we can anticipate what the expression will look like. We know that x is indexed by the integers from 1 to 4 so we know that <span style="color:darkblue; font-family:Courier">j</span> will take on the values from 1 to 4 and we also know that all value of <span style="color:darkblue; font-family:Courier">x</span> will be zero for all indexes so we know that the value of <span style="color:darkblue; font-family:Courier">expr</span> will be something like

In [ ]:
0 + instance.x[1] + instance.x[2] + instance.x[3] + instance.x[4]

The value of <span style="color:darkblue; font-family:Courier">j</span> will be evaluated because it is a Python variable; however, because it is a Pyomo variable, the value of <span style="color:darkblue; font-family:Courier">instance.x[j]</span> not be used, instead the variable object will appear in the expression. That is exactly what we want in this case. When we wanted to use the current value in the <span style="color:darkblue; font-family:Courier">if</span> statement, we used the <span style="color:darkblue; font-family:Courier">value</span> method to get it.

The next line adds to the constaint list called <span style="color:darkblue; font-family:Courier">c</span> the requirement that the expression be greater than or equal to one:

In [ ]:
    instance.c.add( expr >= 1 )

The proof that this precludes the last solution is left as an exerise for the reader.

The final lines in the outer for loop find a solution and display it:

In [ ]:
    results = opt.solve(instance)
    print results

###14.2 Changing the Model or Data and Re-solving
The <span style="color:darkblue; font-family:Courier">iterative1.py</span> example illustrates how a model can be changed and then re-solved. In that example, the model is changed by adding a constraint, but the model could also be changed by altering the values of parameters. Note, however, that in these examples, we make the changes to the <span style="color:darkblue; font-family:Courier">instance</span> object rather than the <span style="color:darkblue; font-family:Courier">model</span> object so that we do not have to create a new <span style="color:darkblue; font-family:Courier">model</span> object. Here is the basic idea:

1 Create an <span style="color:darkblue; font-family:Courier">AbstractModel</span> (suppose it is called <span style="color:darkblue; font-family:Courier">model</span>)

2 Call <span style="color:darkblue; font-family:Courier">model.create_instance()</span> to create an instance (suppose it is called <span style="color:darkblue; font-family:Courier">instance</span>)

Solve instance

Change someting in instance

Call presolve

Solve instance again

If instance has a parameter whose name is in ParamName with an index that is in idx, the the value in NewVal can be assigned to it using